<a href="https://colab.research.google.com/github/torrhen/pytorch/blob/main/05_pytorch_script_mode_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
  import os
  # create new folder for storing modular files
  os.makedirs('script_mode', exist_ok=True)

  !pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# script for downloading and extracting image data sets
%%writefile script_mode/get_data.py
import os
import zipfile # file extracting
from pathlib import Path # file management
import requests # HTTP requests

def download_and_extract_data(url, folder_name):
  # store file paths
  data_path = Path('data/')
  image_path = data_path / folder_name

  # create the image pth folder if it does not exist
  if image_path.is_dir():
    print(f"{image_path} already exists.")
  else:
    image_path.mkdir(parents=True, exist_ok=True)

    # download image zip file from GitHub
    with open(data_path / folder_name / ".zip", 'wb') as f:
      response = requests.get(url)
      f.write(response.content)

    # extract images from image zip file
    with zipfile.ZipFile(data_path / folder_name / ".zip", 'r') as z:
      z.extractall(image_path)

    # delete download image zip file - no longer required
    os.remove(data_path / folder_name / ".zip")

Overwriting script_mode/get_data.py


In [3]:
# script for creating the train and test datasets and data loaders from extracted image folders
%%writefile script_mode/data.py

import os
from torch.utils.data import DataLoader
from torchvision import transforms, datasets

def create_dataloaders(train_dir: str, test_dir: str, transform: transforms.Compose, batch_size: int, num_workers: int = os.cpu_count()):
  # create datasets from image file folder and apply transform
  train_data = datasets.ImageFolder(root=train_dir, transform=transform, target_transform=None)
  test_data = datasets.ImageFolder(root=test_dir, transform=transform, target_transform=None)

  # divide the datasets into batches and shuffle if applicable
  train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
  test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

  return train_loader, test_loader, train_data.classes

Overwriting script_mode/data.py


In [4]:
# script for defining the architecture of the neural network models
%%writefile script_mode/model.py
import torch
import torch.nn as nn

class TinyVGG(nn.Module):
  def __init__(self, input_size, hidden_units, output_size):
    super().__init__()
    self.block1 = nn.Sequential(
        nn.Conv2d(in_channels=input_size, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, stride=1, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.block2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13, out_features=output_size)
    )

  def forward(self, x):
    return self.fc(self.block2(self.block1(x)))




Overwriting script_mode/model.py


In [5]:
# script for training and evaluating a model at every epoch
%%writefile script_mode/train.py
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchmetrics import Accuracy

def train_step(model: nn.Module,
               dataloader: DataLoader,
               loss_fn: nn.Module,
               accuracy_fn: Accuracy,
               optimizer: torch.optim.Optimizer,
               device: torch.device):
  
  train_loss, train_acc = 0.0, 0.0

  model.train()

  for batch, (X, y) in enumerate(dataloader):
    # allocate to device
    X, y = X.to(device), y.to(device)
    # forward pass
    logits = model(X)
    labels = torch.argmax(torch.softmax(logits, dim=1), dim=1)
    # calculate loss
    loss = loss_fn(logits, y)
    train_loss += loss
    # calculate accuracy
    acc = accuracy_fn(labels, y)
    train_acc += acc

    # stop accumulation of gradients
    optimizer.zero_grad()
    # backpropagation
    loss.backward()
    # gradient descent update
    optimizer.step()

  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  return train_loss, train_acc

def test_step(model: nn.Module,
              dataloader: DataLoader,
              loss_fn: nn.Module,
              accuracy_fn: Accuracy,
              device: torch.device):
  
  test_loss, test_acc = 0.0, 0.0

  model.eval()
  with torch.inference_mode():
    for batch, (X, y) in enumerate(dataloader):
      # allocate to device
      X, y = X.to(device), y.to(device)
      # forward pass
      logits = model(X)
      labels = torch.argmax(torch.softmax(logits, dim=1), dim=1)
      # calculate loss
      loss = loss_fn(logits, y)
      test_loss += loss
      # calculate accuracy
      acc = accuracy_fn(labels, y)
      test_acc += acc

    test_loss /= len(dataloader)
    test_acc /= len(dataloader)

  return test_loss, test_acc

def train(model: nn.Module,
          train_loader: DataLoader,
          test_loader: DataLoader,
          loss_fn: nn.Module,
          accuracy_fn: Accuracy,
          optimizer: torch.optim.Optimizer,
          device:  torch.device,
          epochs: int = 5):
  
  results = {'train_loss':[],
             'train_acc':[],
             'test_loss':[],
             'test_acc':[]}

  for epoch in range(epochs):
    # train the model on the training data
    train_loss, train_acc = train_step(model, train_loader, loss_fn, accuracy_fn, optimizer, device)
    # test the model on the test data
    test_loss, test_acc = test_step(model, test_loader, loss_fn, accuracy_fn, device)

    # print results for each epoch
    print(f"Epoch: {epoch+1} | Train Loss: {train_loss:.5f} | Train Accuracy: {train_acc:.2f} | Test Loss: {test_loss:.5f} | Test Accuracy: {test_acc:.2f}")

    # store the results for each epoch
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_acc)
    results['test_loss'].append(test_loss)
    results['test_acc'].append(test_acc)
  
  return results

Overwriting script_mode/train.py


In [23]:
# script for saving and loading trained models
%%writefile script_mode/utils.py
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_folder: str,
               model_name: str):
  
  # store file path to save model
  target_folder_path = Path(target_folder)
  # create folder where model will be saved
  target_folder_path.mkdir(parents=True, exist_ok=True)

  save_path = target_folder_path / model_name
  # save model to save path
  torch.save(obj=model.state_dict(), f=save_path)

Overwriting script_mode/utils.py


In [7]:
# main script for training a neural network
%%writefile script_mode/main.py
import argparse
import torch
import os
from torchvision import transforms
import get_data, data, model, train, utils
from torchmetrics import Accuracy

# parse program arguments
parser = argparse.ArgumentParser(prog='05_pytorch_script_mode_exercises')
parser.add_argument('--TRAIN_FOLDER', action='store', default='data/pizza_steak_sushi/train')
parser.add_argument('--TEST_FOLDER', action='store', default='data/pizza_steak_sushi/test')
parser.add_argument('--LEARNING_RATE', action='store', default=0.001, type=float)
parser.add_argument('--BATCH_SIZE', action='store', default=32, type=int)
parser.add_argument('--NUM_EPOCHS', action='store', default=5, type=int)
parser.add_argument('--HIDDEN_UNITS', action='store', default=10, type=int)
# store the arguments
args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

get_data.download_and_extract_data("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip", "pizza_steak_sushi")

# setup transform to be applied when creating data loaders
transform = transforms.Compose(
    [
        transforms.Resize((64, 64)),
        transforms.ToTensor()
    ]
)
# create data loaders for training and test data
train_loader, test_loader, classes = data.create_dataloaders(args.TRAIN_FOLDER, 
                                                             args.TEST_FOLDER, 
                                                             transform, 
                                                             args.BATCH_SIZE)

# create model
torch.manual_seed(42)
model1 = model.TinyVGG(3, args.HIDDEN_UNITS, 3).to(device) # allocate to device

# create accuracy function
accuracy_fn = Accuracy(task='multiclass', num_classes=3)

# multiclass cross entropy loss
loss_fn = torch.nn.CrossEntropyLoss()
# Adam optimizer
optimizer = torch.optim.Adam(params=model1.parameters(), lr=args.LEARNING_RATE)

# train the model
model1_results = train.train(model1, train_loader, test_loader, loss_fn, accuracy_fn, optimizer, device, args.NUM_EPOCHS)

# save the model
utils.save_model(model1, 'models', 'model1.pth')

Overwriting script_mode/main.py


In [8]:
# run in terminal
!python script_mode/main.py

data/pizza_steak_sushi already exists.
Epoch: 1 | Train Loss: 1.10634 | Train Accuracy: 0.30 | Test Loss: 1.09832 | Test Accuracy: 0.29
Epoch: 2 | Train Loss: 1.09951 | Train Accuracy: 0.33 | Test Loss: 1.06984 | Test Accuracy: 0.54
Epoch: 3 | Train Loss: 1.08625 | Train Accuracy: 0.49 | Test Loss: 1.07999 | Test Accuracy: 0.52
Epoch: 4 | Train Loss: 1.08246 | Train Accuracy: 0.41 | Test Loss: 1.05990 | Test Accuracy: 0.57
Epoch: 5 | Train Loss: 1.06301 | Train Accuracy: 0.41 | Test Loss: 1.06117 | Test Accuracy: 0.55


In [46]:
# script for predicting the label for a custom image
%%writefile script_mode/predict.py
import torch
import torchvision
from torchvision import transforms
import model, utils
from PIL import Image
import argparse
import matplotlib.pyplot as plt
import numpy as np

# read in program arguments
parser = argparse.ArgumentParser(prog='predict the class of a custom image')
parser.add_argument('--image_path', action='store')
args = parser.parse_args()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# display image using PIL
if args.image_path is not None:
  # create tensor of image
  img_tensor = torchvision.io.read_image(str(args.image_path)).type(torch.float32)
  # normalize
  img_tensor /= 255
  # resize image using transform
  resize_transform = transforms.Resize((64, 64))
  img_tensor = resize_transform(img_tensor)
  # add single batch dimension and allocate to device
  img_tensor = img_tensor.unsqueeze(dim=0).to(device)

  # load previously saved model
  new_model = model.TinyVGG(3, 10, 3)
  new_model.load_state_dict(torch.load(f='models/model1.pth'))
  new_model = new_model.to(device)

  new_model.eval()
  with torch.inference_mode():
    prediction_logits = new_model(img_tensor)
    prediction_labels = torch.argmax(torch.softmax(prediction_logits, dim=1), dim=1)

  print(f"Prediction: {prediction_labels.item()}")

Overwriting script_mode/predict.py


In [47]:
!python script_mode/predict.py --image steak.jpeg

Prediction: 2
